In [ ]:
import sqlite3

# Display SQLite version number
print (sqlite3.sqlite_version)

### Connecting to DB

In [ ]:
# Connect to the Database - the DATABASE must already exisit
# Example of path: 'C:\\Users\\nm319\\OneDrive - University of Exeter\\__Exeter Teaching (2020-21)\\Lab Sessions\\Week 3\\XXX.db'
conn=sqlite3.connect('C:\\Users\\nm319\\OneDrive - University of Exeter\\__Exeter Teaching (2020-21)\\Lab Sessions\\Week 4\\BEMM459.db')

# conn is an object of the Connection class - the next command is only for display
print(type(conn))

# The connection object (conn) has access to various methods of the Connection class. 
# We are using the method cursor() and which returns a cursor object.
# The cursor object is essential to perform any operation on the database (CRUD operations).
cur=conn.cursor()

# The next command is only for display
print(type(cur))

### Listing tables

In [ ]:
# Listing all tables in your database (suggest using SQLIte prompt and command .tables)
# Once we have the cursor object, we can perform all SQL operations with the help of execute() method.

# define a function
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

In [ ]:
# call the function and pass connection object 
tables = tables_in_sqlite_db(conn)

#print tables in the current database
print(tables)

### Adding tables using SQL DDL Statement

In [ ]:
# Create a new table called ECC_DEPARTMENT
# Note: If the table is created succesfully, executing the code again will lead to exception

# Once we have the cursor object, we can perform all SQL operations with the help of .execute() method

qry='''
CREATE TABLE ECC_DEPARTMENT (
departmentID INTEGER PRIMARY KEY,
departmentName TEXT (30),
departmentAddress TEXT (40)
);
'''
try:
        cur.execute(qry)
        print ('Table created successfully')
except:
# If table already exists then use the SQLite console to connect to the database (BEMM459.db) and then use the drop table command.
# .. Altenatively, use the SQLiteStudio GUI to delete table ECC_DEPARTMENT and execute this code block again.
        print ('Error in creating table')
        
        # Use the next statement with extreme caution as both table and data will be lost
        cur.execute("DROP TABLE IF EXISTS ECC_DEPARTMENT;")

In [ ]:
# Create four tables using the method executescript() that is defined in the cursor class. 
# executescript() makes it possible to execute multiple SQL statements.

qry='''
CREATE TABLE ECC_PROGRAMME (
        programmeID INTEGER   PRIMARY KEY,
        programmeName   TEXT (40),
        programmeDepartmentID INTEGER REFERENCES ECC_DEPARTMENT(departmentID)
    );
CREATE TABLE ECC_STUDENT (
        studentID INTEGER   PRIMARY KEY AUTOINCREMENT,
        studentFirstName   TEXT (30),
        studentLastName   TEXT (30),
        studentProgramme INTEGER REFERENCES PROGRAMME(programmeID)
    );
CREATE TABLE ECC_MODULE (
        moduleID     INTEGER     PRIMARY KEY,
        moduleName TEXT (30)
    );
CREATE TABLE ECC_PROGRAMME_MODULE (
        programmemoduleID     INTEGER     PRIMARY KEY AUTOINCREMENT,
        programmemoduleProgrammeID    INTEGER   REFERENCES ECC_PROGRAMME (programmeID),
        programmemoduleModuleID INTEGER  REFERENCES ECC_MODULE (moduleID),
        programmemoduleCore TEXT (10)  
    );
'''
try:
        cur.executescript(qry)
        print ('Tables created successfully')
except:
        print ('Error in creating tables')
        
        # Use the next statement with extreme caution as both table and data will be lost
        # cur.execute("DROP TABLE IF EXISTS ECC_PROGRAMME;")
        # cur.execute("DROP TABLE IF EXISTS ECC_STUDENT;")
        # cur.execute("DROP TABLE IF EXISTS ECC_MODULE;")
        # cur.execute("DROP TABLE IF EXISTS ECC_PROGRAMME_MODULE;")

In [ ]:
# check to see if the new table is created
# call the function and pass connection object 
tables = tables_in_sqlite_db(conn)

#print tables in the current database
print(tables)

### Inserting records using SQL DML Statement

In [ ]:
# Insert one record in table ECC_DEPARTMENT
# Commit only if no exception is encountered, else rollback

qry="insert into ECC_DEPARTMENT values (1, 'Professional Services', '19 Orchid Road, Exeter EX3 1GT');"

try:
        cur.execute(qry)
        conn.commit()
        print ('One record inserted successfully..commit')
except:
        print ('Error in insert operation..rollback')
        conn.rollback()
        print(conn)

In [ ]:
# Insert multiple records in table ECC_DEPARTMENT

qry="insert into ECC_DEPARTMENT (departmentID, departmentName, departmentAddress) values (?,?,?);"
collegelist=[(10,'College of Business','11 King Edwart Street, Exeter, EX6 7ED'),
           (20, 'College of Management','13 King Edwart Street, Exeter, EX6 7ED'),
           (30,'College of Accounting & Finance', '51 Rosebloom Avenue, Exeter, EX1 5RT'),
           (40,'College of Economics', '116 Exeter High Street, Exeter, EX4 5TY')]

try:
        cur.executemany(qry, collegelist)
        conn.commit()
        print ('Records inserted successfully..committed')
except:
        print ('Error in insert operation..rollback')
        conn.rollback()

In [ ]:
# Insert one record in table ECC_DEPARTMENT based on user input

inputDepartmentNum=int(input('Enter ECC_department number:'))
inputName=input('Enter name of ECC_department to update:')
inputAddress=input('Enter name of ECC_department address:')

qry="insert into ECC_DEPARTMENT values (?,?,?);"

try:
        cur.execute(qry, (inputDepartmentNum,inputName, inputAddress))
        print ('New department added')
        conn.commit()
except:
        print ('Error in adding department .. rollback')
        conn.rollback()


### Querying data

In [ ]:
# Query and display one record from the table ECC_DEPARTMENT

# Prepare the query String
qry="select * from ECC_DEPARTMENT;"

# Execute query on SQLite
cur.execute(qry)

# Fetch and display one row
row=cur.fetchone()

print (row)

In [ ]:
# Query and display records from the table ECC_DEPARTMENT (all rows)

# Prepare the query String
qry="select * from ECC_DEPARTMENT;"

# Execute query on SQLite
cur.execute(qry)

# Fetch and display all rows
rows=cur.fetchall()

for row in rows:
    print (row)

In [ ]:
# Query and display records from the table ECC_DEPARTMENT based on user input

inputDepartmentNum=input ('Enter ECE_department number:')
qry="select * from ECC_department where departmentID=?";
cur.execute(qry, (inputDepartmentNum,))
row=cur.fetchone()
print (row)

# IMPORTANT: Individual items in the tuple can be accessed by index .. we are accessing the second attribute
print ("Department Name is ", row[1])

### Updating data using SQL DML Statement

In [ ]:
# Update table based on user input

inputName=input('Enter name of ECC_department to update:')
inputDepartmentNum=int(input('Enter new department number:'))
qry='update ECC_department set departmentID=? where departmentName=?'

try:
        cur.execute(qry, (inputDepartmentNum,inputName))
        print ('Department name updated')
        conn.commit()
except:
        print ('Error in update operation .. rollback')
        conn.rollback()

### Deleting data using SQL DML Statement

In [ ]:
# Delete record based on user input

inputName=input('Enter name of ECC_department to delete:')
qry='delete from ECC_department where departmentName=?'
try:
        cur.execute(qry, (inputName,))
        print ('Department deleted')
        conn.commit()
except:
        print ('Error in deleting department', inputName)
        conn.rollback()


# Database Backup and Restore

### Database backup

In [ ]:
# Creating databse dump ... we are connected to BEMM459.db
# If conn object is closed then uncomment the next line and execute code
# conn=sqlite3.connect('C:\\Users\\nm319\\OneDrive - University of Exeter\\__Exeter Teaching (2020-21)\\Lab Sessions\\Week 4\\BEMM459.db')


file=open('BEMM459_backup.sql','w')

for line in conn.iterdump():
        file.write('{}\n'.format(line))
        
file.close()

# Closing database connection
# conn.close()

### Database restore

In [ ]:
# Creating new database, reading content of the dump file and executing SQL statements in it using cursor object's executescript() method

connRestore=sqlite3.connect('C:\\Users\\nm319\\OneDrive - University of Exeter\\__Exeter Teaching (2020-21)\\Lab Sessions\\Week 4\\BEMM459_BKUP.db')

file=open('BEMM459_backup.sql','r')
qry=file.read()
file.close()

curRestore=connRestore.cursor()
curRestore.executescript(qry)

# call function (defined earlier) and pass connection object 
tables = tables_in_sqlite_db(connRestore)

#print tables in the newly restored database
print(tables)

connRestore.close()

In [ ]:
# Close database connection to BEMM459.db
conn.close()